In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bgu-recsys-ctr-prediction/submission_example.csv
/kaggle/input/bgu-recsys-ctr-prediction/test_file.csv
/kaggle/input/bgu-recsys-ctr-prediction/Taboola Dataset Notice.docx
/kaggle/input/bgu-recsys-ctr-prediction/train_data/train5/part-00045.csv
/kaggle/input/bgu-recsys-ctr-prediction/train_data/train5/part-00043.csv
/kaggle/input/bgu-recsys-ctr-prediction/train_data/train5/part-00049.csv
/kaggle/input/bgu-recsys-ctr-prediction/train_data/train5/part-00044.csv
/kaggle/input/bgu-recsys-ctr-prediction/train_data/train5/part-00050.csv
/kaggle/input/bgu-recsys-ctr-prediction/train_data/train5/part-00041.csv
/kaggle/input/bgu-recsys-ctr-prediction/train_data/train5/part-00047.csv
/kaggle/input/bgu-recsys-ctr-prediction/train_data/train5/part-00042.csv
/kaggle/input/bgu-recsys-ctr-prediction/train_data/train5/part-00048.csv
/kaggle/input/bgu-recsys-ctr-prediction/train_data/train5/part-00046.csv
/kaggle/input/bgu-recsys-ctr-prediction/train_data/train1/part-00008.csv
/kaggle/inpu

## Installations & Imports

In [2]:
# Downloading deepFm model
!pip install deepctr[gpu]

     |████████████████████████████████| 116 kB 1.3 MB/s 
     |████████████████████████████████| 454.3 MB 14 kB/s 
     |████████████████████████████████| 462 kB 35.4 MB/s 
     |████████████████████████████████| 1.2 MB 52.8 MB/s 
     |████████████████████████████████| 6.0 MB 39.0 MB/s 
     |████████████████████████████████| 4.0 MB 34.3 MB/s 
     |████████████████████████████████| 394.3 MB 12 kB/s 


In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat, DenseFeat,VarLenSparseFeat,get_feature_names

from tensorflow.python.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import glob
import datetime
from sklearn import preprocessing 

## Read Data

In [4]:
GLOBAL_PATH_TRAIN = "../input/bgu-recsys-ctr-prediction/train_data/*/*"
TEST_PATH = "../input/bgu-recsys-ctr-prediction/test_file.csv"

In [5]:
# Reading train data 
## We read only 20 files from the entire data 

from tqdm import tqdm
all_paths = glob.glob(GLOBAL_PATH_TRAIN)

data = pd.DataFrame()
for path in tqdm(all_paths[:20]):
    data = data.append(pd.read_csv(path))

100%|██████████| 20/20 [04:28<00:00, 13.45s/it]


In [6]:
# reading test data 
test = pd.read_csv(TEST_PATH)


In [7]:
# We wanted to apply the entire preprocess on all the data at once 
# so we added the missing columns in each set to have a defualt value
data["Id"] = -1
test["is_click"] = -1

In [8]:
# Combining the data to run the preprocess on the entire set
data_combined = pd.concat([test,data])


## Data preprocessing

In [9]:
# The columns we want to delete
delete_coulmns = ["gmt_offset","user_id_hash"]
# Our multi-value features
multi_value = ["target_item_taxonomy"]
# Sparse features
sparse_features  = ["target_id_hash","syndicator_id_hash","campaign_id_hash","placement_id_hash","publisher_id_hash","source_id_hash","source_item_type","browser_platform","country_code","region"]
# target column
target = ['is_click']


In [10]:
# This is done so the ram won't collapse the run
del data
del test

In [11]:
# Creating dense features list by substracting all other features
dense_features  = list(set(data_combined.columns) - set(target) - set(sparse_features) - set(multi_value) - set(delete_coulmns) - set(["Id"]))
data_combined = data_combined.drop(delete_coulmns, axis = 1)

In [12]:
# Data preprocessing 
# Sparse data is transformed to one hot vectors using the label encoder
for feat in sparse_features:
    lbe = LabelEncoder()
    data_combined[feat] = lbe.fit_transform(data_combined[feat])

# Dense features are normalized to the range of 0,1
mms = MinMaxScaler(feature_range=(0,1))
data_combined[dense_features] = mms.fit_transform(data_combined[dense_features])


In [13]:
# This function handles the multi-value feature column.
# It seperates the string into all the a list of values. 
def split(x):
    key_ans = x.split('~')
    for key in key_ans:
        if key not in key2index:
            # Notice : input value 0 is a special "padding",so we do not use 0 to encode valid feature for sequence input
            key2index[key] = len(key2index) + 1
    return list(map(lambda x: key2index[x], key_ans))

### Multi Value Features

In [14]:
# Extracting all values using the split function. 
# We will later use this list to create the multi-value embedding vectors
key2index = {}
genres_list = list(map(split, data_combined[multi_value[0]].values))
genres_length = np.array(list(map(len, genres_list)))
max_len = max(genres_length)
# Notice : padding=`post`
genres_list = pad_sequences(genres_list, maxlen=max_len, padding='post', )

In [15]:
# Creating the features for the model
# We create the embeddings for the sparse and multi-value featrues and combine the features into features list
fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data_combined[feat].max() + 1,embedding_dim=3)
                       for i,feat in enumerate(sparse_features)] + [DenseFeat(feat, 1,)
                      for feat in dense_features]

varlen_feature_columns = [VarLenSparseFeat(SparseFeat(multi_value[0],vocabulary_size= len(
            key2index) + 1,embedding_dim=3), maxlen=max_len, combiner='mean',weight_name=None)]  # Notice : value 0 is for padding for sequence input feature


In [16]:
# This part decides which features will be inserted into the linear part of the model and which to the dnn. 
# We decided that all features will be inserted to both. 
linear_feature_columns = fixlen_feature_columns + varlen_feature_columns
dnn_feature_columns = fixlen_feature_columns + varlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

## The Model

In [17]:
# Splitting the data back into train and test
train = data_combined[data_combined["Id"] == -1]
test = data_combined[data_combined["is_click"] == -1]

# Creating the input that will be inserted into the model
train_model_input = {name:train[name].values for name in feature_names}
train_model_input[multi_value[0]] = genres_list[train.index]
test_model_input = {name:test[name].values for name in feature_names}
test_model_input[multi_value[0]] = genres_list[test.index]

target_values = train[target].values

# The model
model = DeepFM(linear_feature_columns,dnn_feature_columns,task='binary',
               dnn_hidden_units = (256, 256, 128))
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=opt,metrics=[tf.keras.metrics.AUC(), "accuracy"])



In [18]:
history = model.fit(train_model_input, target_values,
                    batch_size=256, epochs=10, verbose=2, validation_split=0.2, )


Epoch 1/10
27348/27348 - 512s - loss: 0.4798 - auc: 0.8509 - accuracy: 0.7724 - val_loss: 0.4745 - val_auc: 0.8559 - val_accuracy: 0.7764
Epoch 2/10
27348/27348 - 512s - loss: 0.4733 - auc: 0.8562 - accuracy: 0.7774 - val_loss: 0.4724 - val_auc: 0.8568 - val_accuracy: 0.7777
Epoch 3/10
27348/27348 - 497s - loss: 0.4721 - auc: 0.8570 - accuracy: 0.7782 - val_loss: 0.4719 - val_auc: 0.8572 - val_accuracy: 0.7786
Epoch 4/10
27348/27348 - 500s - loss: 0.4714 - auc: 0.8575 - accuracy: 0.7786 - val_loss: 0.4713 - val_auc: 0.8576 - val_accuracy: 0.7788
Epoch 5/10
27348/27348 - 495s - loss: 0.4709 - auc: 0.8578 - accuracy: 0.7791 - val_loss: 0.4717 - val_auc: 0.8576 - val_accuracy: 0.7784
Epoch 6/10
27348/27348 - 506s - loss: 0.4706 - auc: 0.8580 - accuracy: 0.7792 - val_loss: 0.4750 - val_auc: 0.8569 - val_accuracy: 0.7768
Epoch 7/10
27348/27348 - 497s - loss: 0.4704 - auc: 0.8582 - accuracy: 0.7793 - val_loss: 0.4706 - val_auc: 0.8581 - val_accuracy: 0.7791
Epoch 8/10
27348/27348 - 497s - lo

In [19]:
pred_ans = model.predict(test_model_input, batch_size=256)

In [20]:
pred_df = pd.DataFrame(pred_ans)
pred_df.reset_index(inplace=True)
pred_df.columns = ['Id','Predicted']
pred_df.to_csv('my_submission5.csv',index=False)